# Fit an LSTM (Model 1) to Shasta Reservoir

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

## Data Processing

In [2]:
from src.data.data_processing import *
from src.data.data_fetching import *

In [3]:
# Read in data, columns are [inflow, outflow, storage]
df = usbr_fetch_data(name='Shasta', vars=['inflow', 'outflow', 'storage'])

In [5]:
# Add day of the year (doy) as another column
df['doy'] = df.index.to_series().dt.dayofyear

In [8]:
# Select data window (beginning of 1944 to end of 2022)
df = df['1944-01-01':'2022-12-31'].copy()

In [9]:
# Run data processing pipeline
pipeline = processing_pipeline(train_frac=0.6, val_frac=0.2, test_frac=0.2, chunk_size=3*365, pad_value=-1)
ts_train, ts_val, ts_test = pipeline.process_data(df) # output tensors of shape (#chunks, chunk_size/timestep, [inflow, outflow, storage, doy])

In [14]:
## Separate inputs(X) and targets (y)

# select inflow and doy as input features
X_train, X_val, X_test = ts_train[:, :, [0, 3]], ts_val[:, :, [0, 3]], ts_test[:, :, [0, 3]]
# select outflow as target feature
y_train, y_val, y_test = ts_train[:, :, [1]], ts_val[:, :, [1]], ts_test[:, :, [1]]